In [1]:
import requests
import netCDF4 as nc
import re
import os
import numpy as np
import simplekml
from tabulate import tabulate

def checkOKStatusRequest (response):
 if (response.status_code == 200): return True
 else: return False

APIKey = "eyJvcmciOiI1ZTU1NGUxOTI3NGE5NjAwMDEyYTNlYjEiLCJpZCI6IjI4ZWZlOTZkNDk2ZjQ3ZmE5YjMzNWY5NDU3NWQyMzViIiwiaCI6Im11cm11cjEyOCJ9" #Public key

In [2]:

def getDownloadURL (year, month, day, hour, minute):
	fileName = f"KMDS__OPER_P___10M_OBS_L2_{year}{month}{day}{hour}{minute}.nc"
	print(f"Fetching file {fileName}")

	try:
		response = requests.get(f"https://api.dataplatform.knmi.nl/open-data/v1/datasets/Actuele10mindataKNMIstations/versions/2/files/{fileName}/url", headers={"Authorization": f"Bearer {APIKey}"})
	except:
		print("Can't fetch URL (error in syntax probably)")
		return

	if (checkOKStatusRequest(response) == False):
		print(f"Fetching URL failed with status code {response.status_code}, body response:")
		print(response.content)
		return

	if (response.content):
		lastModified = response.json()["lastModified"]
		print(f"Fetched temporary download URL, file last modified {lastModified}")
		return response.json()["temporaryDownloadUrl"]


# print(getDownloadURL(2023, "01", "19", "14", "20"))


In [11]:
kml = simplekml.Kml()
table = []

def runFunc():
	
	# All UTC dates and times
	year = 2023
	month = "02"
	day = "17"
	hour = "21"
	minute = "30"
	
	url = getDownloadURL(year, month, day, hour, minute)

	print(f"Parsing measurements from {year}-{month}-{day}T{hour}:{minute}:00+00:00")

	try:
		response = requests.get(url)
	except:
		print("Can't fetch URL (error in syntax probably)")
		return

	if (checkOKStatusRequest(response) == False):
		print(f"Fetching URL failed with status code {response.status_code}, body response:")
		print(response.content)
		return

	contentDisposition = response.headers["content-disposition"]
	fileName = re.findall("filename=\"(.+)\"", contentDisposition)[0]
	path = f"netCDF files/{fileName}"

	try:
			if(os.path.exists(path)): os.remove(path)
			open(path, "wb").write(response.content)
	except:
		print(f"Saving file {fileName} failed")
		return
 
	dataset = nc.Dataset(path)
	staionsIDs = dataset["station"][:]

	# for variable in dataset.variables:
	# 	print(variable)

	for stationID in staionsIDs:
		indexStation = np.where(staionsIDs == stationID)[0][0]

		windSpeed = dataset["ff"][indexStation][0]
		windGusts = dataset["gff"][indexStation][0]
		windDirection = dataset["dd"][indexStation][0]


		if (windSpeed == dataset["ff"]._FillValue or isinstance(windSpeed, float) != True): windSpeed = -999
		if (windGusts == dataset["gff"]._FillValue or isinstance(windGusts, float) != True): windGusts = -999
		if (windDirection == dataset["dd"]._FillValue or isinstance(windDirection, float) != True): windDirection = -999

		windSpeed = round(windSpeed, 2)
		windGusts = round(windGusts, 2)
		windDirection = int(round(windDirection, 0))

		# Log rows, better: open generated HTML
		# print(stationID, windSpeed, windGusts, windDirection)

		stationName = dataset["stationname"][indexStation]
		stationLat = dataset["lat"][indexStation]
		stationLon = dataset["lon"][indexStation]

		kml.newpoint(name=f"{stationID}, {stationName}", coords=[( stationLon, stationLat)])
		table.append([stationID, stationName, stationLat, stationLon, windSpeed, windGusts, windDirection])
		
	#Save KML file with only the metadata of the locations
	kml.save("locationsNetCDF.kml")

	# Make a table for the measured values
	f = open("parsingNetCDFResult.html", "w")
	tableHTMl = tabulate(table, ["ID", "Name", "Lat", "Lon", "Windspeed (m/s)", "Windgusts (m/s)", "Winddirection (degrees)"], tablefmt="html")
	f.write(f"<h1>{fileName}</h1><br>{tableHTMl}")
	f.close()

	dataset.close()
	print("Parsing done, open generated HTML file!")
	
runFunc()

Fetching file KMDS__OPER_P___10M_OBS_L2_202302172130.nc
Fetched temporary download URL, file last modified 2023-02-17T22:09:49+00:00
Parsing measurements from 2023-02-17T21:30:00+00:00
Parsing done, open generated HTML file!
